In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import GridSearchCV

In [2]:
Churn_data=pd.read_csv("D:\Study\Python\scripts\Deep_Learning\Banking_Customer_Churn_Modeling\Churn_Modelling.csv")

In [3]:
Churn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [4]:
Churn_data.shape

(10000, 14)

In [5]:
Churn_data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
Churn_data.duplicated().sum()

0

In [7]:
Churn_data['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [8]:
Churn_data['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [9]:
Churn_data['Gender']=pd.get_dummies(Churn_data['Gender'], drop_first=True)

In [10]:
Geography=pd.get_dummies(Churn_data['Geography'], drop_first=True)

In [11]:
Churn_data=pd.concat([Churn_data, Geography], axis=1)

In [12]:
Churn_data=Churn_data.drop(['Geography'], axis=1)

In [13]:
Churn_data=Churn_data.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [14]:
Churn_data.head(10)

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0          619       0   42       2       0.00              1          1   
1          608       0   41       1   83807.86              1          0   
2          502       0   42       8  159660.80              3          1   
3          699       0   39       1       0.00              2          0   
4          850       0   43       2  125510.82              1          1   
5          645       1   44       8  113755.78              2          1   
6          822       1   50       7       0.00              2          1   
7          376       0   29       4  115046.74              4          1   
8          501       1   44       4  142051.07              2          0   
9          684       1   27       2  134603.88              1          1   

   IsActiveMember  EstimatedSalary  Exited  Germany  Spain  
0               1        101348.88       1        0      0  
1               1        112542.58       0        0      1  
2               0        113931.57       1        0      0  
3               0         93826.63       0        0      0  
4               1         79084.10       0        0      1  
5               0        149756.71       1        0      1  
6               1         10062.80       0        0      0  
7               0        119346.88       1        1      0  
8               1         74940.50       0        0      0  
9               1         71725.73       0        0      0

In [15]:
X=Churn_data.drop(['Exited'], axis=1)
Y=Churn_data['Exited']

In [16]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=0.2, random_state=0)

In [18]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.activations import relu, sigmoid

# Hyperparameter Tuning to decide number of Neurons and hidden layers:

In [19]:
def fun(layers, activation):
    model=Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes, input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
estimator=KerasClassifier(build_fn=fun, verbose=0)

layers=[(20,),(20,15),(40,30,20)]
activations=['relu','sigmoid']
param_grid=dict(layers=layers, activation=activations, batch_size=[128,256], epochs=[30])
grid=GridSearchCV(estimator=estimator, param_grid=param_grid, cv=5)

In [20]:
grid_result=grid.fit(X_train,Y_train)

In [21]:
print(grid_result.best_score_, grid.best_params_)

0.8549999952316284 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (40, 30, 20)}


In [24]:
grid_result_predict=grid_result.predict(X_test)
grid_result_predict=grid_result_predict>0.6

In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [26]:
cm=confusion_matrix(Y_test, grid_result_predict)
cm

array([[1549,   46],
       [ 222,  183]], dtype=int64)

In [28]:
accuracy_score(Y_test,grid_result_predict)

0.866